In [1]:
import gdelt

In [2]:
gd = gdelt.gdelt(version=2)

In [7]:
results = gd.Search(['2016 Oct 16','2016 Oct 29'],coverage=True)

/Users/linwood/anaconda2/envs/gdelt/lib/python3.5/site-packages/gdelt/dateFuncs.py:187: UserWarning: 

This query will download 960 files, and likely exhaust your memory with possibly 10s of
                    GBs of data in this single query.  Hit Ctr-C to kill this query if you do not
                    want to continue.
  want to continue.'''.format(len(converted.tolist())))
/Users/linwood/anaconda2/envs/gdelt/lib/python3.5/site-packages/gdelt/parallel.py:37: UserWarning: GDELT did not return data for date time 20161029193000
  warnings.warn(message)
/Users/linwood/anaconda2/envs/gdelt/lib/python3.5/site-packages/gdelt/parallel.py:37: UserWarning: GDELT did not return data for date time 20161029194500
  warnings.warn(message)
/Users/linwood/anaconda2/envs/gdelt/lib/python3.5/site-packages/gdelt/parallel.py:37: UserWarning: GDELT did not return data for date time 20161029200000
  warnings.warn(message)
/Users/linwood/anaconda2/envs/gdelt/lib/python3.5/site-packages/gdelt/parallel.p

In [8]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038953 entries, 0 to 2038952
Data columns (total 61 columns):
GLOBALEVENTID            int64
SQLDATE                  int64
MonthYear                int64
Year                     int64
FractionDate             float64
Actor1Code               object
Actor1Name               object
Actor1CountryCode        object
Actor1KnownGroupCode     object
Actor1EthnicCode         object
Actor1Religion1Code      object
Actor1Religion2Code      object
Actor1Type1Code          object
Actor1Type2Code          object
Actor1Type3Code          object
Actor2Code               object
Actor2Name               object
Actor2CountryCode        object
Actor2KnownGroupCode     object
Actor2EthnicCode         object
Actor2Religion1Code      object
Actor2Religion2Code      object
Actor2Type1Code          object
Actor2Type2Code          object
Actor2Type3Code          object
IsRootEvent              int64
EventCode                object
EventBaseCode            ob